In [4]:
%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.plot import *
from hossam.tensor import *
from pandas import read_excel, pivot_table, melt, to_datetime
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = read_excel("통합6.xlsx")
df

,회원카드번호,연월,미수액,FC 강사,등록명,일일입장,분실,부가서비스사용,레슨여부,매출액,...,일일입장매출,양도비매출,필라테스매출,환불사용금매출,의류매출,분실매출,GX매출,상환액매출,입회비매출,예치금매출
0,12131325,2020-09,0,홍진석,헬스,0,0,0,0,899000,...,0,0,0,0,0,0,0,0,0,0
1,13110008,2021-05,0,신지연,골프,0,0,2,0,1700000,...,0,0,0,0,0,0,0,0,0,0
2,13110008,2021-06,0,신지연,NaN,0,0,0,1,200000,...,0,0,0,0,0,0,0,0,0,0
3,13110008,2022-06,0,신지연,골프,0,0,2,0,1700000,...,0,0,0,0,0,0,0,0,0,0
4,13110008,2023-08,0,신지연,NaN,0,0,1,0,220000,...,0,0,0,0,220000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10025,23120077,2023-12,0,NaN,헬스,0,0,0,0,990000,...,0,0,0,0,0,0,0,0,0,0
10026,24010001,2023-12,0,NaN,헬스,0,0,0,0,990000,...,0,0,0,0,0,0,0,0,0,0
10027,24010002,2023-12,0,NaN,NaN,0,0,0,0,990000,...,0,0,0,0,0,0,0,990000,0,0
10028,24010003,2022-01,0,NaN,헬스,0,0,1,0,1035000,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df1 = df.drop(["미수액","FC 강사","일일입장","분실","부가서비스사용","레슨여부","매출액","일일입장매출","양도비매출","필라테스매출","환불사용금매출","의류매출","분실매출","GX매출","상환액매출","입회비매출","예치금매출","골프전환","단체회원","프로모션","단체명","패스","골프매출","라카매출","헬스매출","PT매출","골프레슨매출"], axis=1)
df1

,회원카드번호,연월,등록명,개월수
0,12131325,2020-09,헬스,12
1,13110008,2021-05,골프,12
2,13110008,2021-06,NaN,0
3,13110008,2022-06,골프,12
4,13110008,2023-08,NaN,0
...,...,...,...,...
10025,23120077,2023-12,헬스,10
10026,24010001,2023-12,헬스,10
10027,24010002,2023-12,NaN,0
10028,24010003,2022-01,헬스,10


In [4]:
# 등록명이 '헬스' 또는 '골프'인 행들만 선택
df2 = df1[df1['등록명'].isin(['헬스', '골프'])]

print(df2)

         회원카드번호       연월 등록명  개월수
0      12131325  2020-09  헬스   12
1      13110008  2021-05  골프   12
3      13110008  2022-06  골프   12
5      13110008  2023-11  골프   12
6      13110009  2020-10  헬스   12
...         ...      ...  ..  ...
10024  23120076  2023-12  헬스   10
10025  23120077  2023-12  헬스   10
10026  24010001  2023-12  헬스   10
10028  24010003  2022-01  헬스   10
10029  88017624  2023-02  골프    3

[6306 rows x 4 columns]


In [5]:
merge1 = read_excel('카드번호출석.xlsx')
merge1

,회원카드번호,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
0,20010013,10.0,4.0,NaN,6.0,2.0,6.0,2,7.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14120047,8.0,4.0,5.0,11.0,13.0,10.0,10,8.0,7.0,...,16.0,9.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,14060030,4.0,6.0,2.0,NaN,7.0,5.0,2,2.0,NaN,...,5.0,4.0,1.0,NaN,3.0,10.0,1.0,NaN,NaN,NaN
3,15120030,13.0,14.0,NaN,3.0,23.0,25.0,25,18.0,2.0,...,7.0,5.0,11.0,7.0,4.0,NaN,2.0,4.0,NaN,NaN
4,19050015,15.0,2.0,2.0,NaN,7.0,12.0,8,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,14010231,4.0,1.0,3.0,NaN,4.0,5.0,7,10.0,1.0,...,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.0,4.0,5.0
1014,14080034,5.0,8.0,2.0,4.0,10.0,3.0,5,1.0,1.0,...,NaN,6.0,5.0,4.0,6.0,7.0,2.0,5.0,3.0,3.0
1015,18030022,10.0,7.0,1.0,2.0,13.0,2.0,7,7.0,4.0,...,18.0,4.0,14.0,9.0,7.0,2.0,8.0,7.0,11.0,12.0
1016,15010006,20.0,20.0,16.0,17.0,19.0,21.0,15,17.0,12.0,...,18.0,16.0,23.0,15.0,4.0,NaN,12.0,14.0,17.0,15.0


In [6]:
df3 = pd.merge(df1, merge1, on='회원카드번호', how='inner')
df3 

,회원카드번호,연월,등록명,개월수,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
0,12131325,2020-09,헬스,12,15.0,14.0,NaN,NaN,NaN,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13110008,2021-05,골프,12,19.0,20.0,14.0,14.0,14.0,19.0,...,17.0,17.0,16.0,19.0,21.0,20.0,13.0,16.0,15.0,19.0
2,13110008,2021-06,NaN,0,19.0,20.0,14.0,14.0,14.0,19.0,...,17.0,17.0,16.0,19.0,21.0,20.0,13.0,16.0,15.0,19.0
3,13110008,2022-06,골프,12,19.0,20.0,14.0,14.0,14.0,19.0,...,17.0,17.0,16.0,19.0,21.0,20.0,13.0,16.0,15.0,19.0
4,13110008,2023-08,NaN,0,19.0,20.0,14.0,14.0,14.0,19.0,...,17.0,17.0,16.0,19.0,21.0,20.0,13.0,16.0,15.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3743,20070073,2021-03,골프,6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3744,20070073,2021-12,헬스,9,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3745,20070073,2022-03,골프,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3746,20070073,2023-06,헬스,12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# 데이터프레임 df3
df = df3

# '연월' 열을 기준으로 오름차순 정렬
df = df.sort_values(by='연월')

# 반복문을 사용하여 각 행에 대해 처리
새로운_데이터프레임들 = []

for index, row in df.iterrows():
    연월 = row['연월']
    개월수 = row['개월수']
    시작열 = df.columns.get_loc(연월)
    끝열 = df.columns.get_loc(연월) + 개월수  # 끝 열 인덱스 수정
    남길열 = df.columns[시작열:끝열]
    
    # NaN 값을 0으로 대체
    row[남길열] = row[남길열].fillna(0)
    
    새로운_df = pd.DataFrame(row).transpose()[['회원카드번호', '연월', '등록명', '개월수'] + list(남길열)]
    새로운_데이터프레임들.append(새로운_df)

# 결과를 하나의 데이터프레임으로 합치기
결과_df = pd.concat(새로운_데이터프레임들, ignore_index=True)

print(결과_df)

        회원카드번호       연월  등록명 개월수 2020-01 2020-02 2020-03 2020-04 2020-05  \
0     17030054  2020-01  NaN   0     NaN     NaN     NaN     NaN     NaN   
1     14060030  2020-01   헬스  12     4.0     6.0     2.0     0.0     7.0   
2     15030037  2020-01   골프  12     7.0    10.0     9.0     4.0    12.0   
3     19070039  2020-01  NaN   0     NaN     NaN     NaN     NaN     NaN   
4     19100068  2020-01  NaN   0     NaN     NaN     NaN     NaN     NaN   
...        ...      ...  ...  ..     ...     ...     ...     ...     ...   
3743  17020088  2023-12  NaN   0     NaN     NaN     NaN     NaN     NaN   
3744  14070134  2023-12   헬스  10     NaN     NaN     NaN     NaN     NaN   
3745  18030038  2023-12   헬스  12     NaN     NaN     NaN     NaN     NaN   
3746  13110114  2023-12   헬스  12     NaN     NaN     NaN     NaN     NaN   
3747  15060067  2023-12   헬스  12     NaN     NaN     NaN     NaN     NaN   

     2020-06  ... 2023-03 2023-04 2023-05 2023-06 2023-07 2023-08 2023-09  \
0        N

In [8]:
# 데이터프레임 df3
df = 결과_df

# '연월' 열을 기준으로 오름차순 정렬
df = df.sort_values(by='연월')

# 각 연월별로 데이터를 담을 딕셔너리
연월별_데이터 = {}

# 반복문을 사용하여 각 연월별로 데이터를 나누어 딕셔너리에 저장
for 연월, 그룹 in df.groupby('연월'):
    연월별_데이터[연월] = 그룹

# 각 연월별 데이터 출력
for 연월, 데이터 in 연월별_데이터.items():
    print(f"{연월}:")
    print(데이터)

2020-01:
       회원카드번호       연월  등록명 개월수 2020-01 2020-02 2020-03 2020-04 2020-05  \
0    17030054  2020-01  NaN   0     NaN     NaN     NaN     NaN     NaN   
131  20010025  2020-01  NaN   0     NaN     NaN     NaN     NaN     NaN   
132  13110097  2020-01   헬스  12    15.0    19.0     2.0     0.0    15.0   
133  18050033  2020-01  NaN   0     NaN     NaN     NaN     NaN     NaN   
134  15070020  2020-01  NaN   0     NaN     NaN     NaN     NaN     NaN   
..        ...      ...  ...  ..     ...     ...     ...     ...     ...   
71   19010007  2020-01   헬스  12     9.0    14.0    13.0    10.0    12.0   
72   20010078  2020-01   헬스  12     1.0    14.0    14.0     5.0    12.0   
74   19010076  2020-01   골프  12     4.0    14.0     6.0    12.0     9.0   
73   18080017  2020-01   헬스  12    22.0    20.0     9.0     0.0    18.0   
64   18010137  2020-01   헬스  12     8.0     5.0     0.0     1.0     0.0   

    2020-06  ... 2023-03 2023-04 2023-05 2023-06 2023-07 2023-08 2023-09  \
0       NaN  .

In [11]:
# # 결과 데이터프레임 df
# df = 결과_df

# # '연월' 열을 기준으로 오름차순 정렬
# df = df.sort_values(by='연월')

# # 결과를 저장할 디렉토리 생성
# 저장_디렉토리 = "월별_데이터"
# if not os.path.exists(저장_디렉토리):
#     os.makedirs(저장_디렉토리)

# # 각 연월별로 데이터를 엑셀 파일로 저장
# for 연월, 그룹 in df.groupby('연월'):
#     파일명 = os.path.join(저장_디렉토리, f"{연월}.xlsx")
#     그룹.to_excel(파일명, index=False)
#     print(f"{연월} 데이터를 {파일명}에 저장했습니다.")

2020-01 데이터를 월별_데이터\2020-01.xlsx에 저장했습니다.
2020-02 데이터를 월별_데이터\2020-02.xlsx에 저장했습니다.
2020-03 데이터를 월별_데이터\2020-03.xlsx에 저장했습니다.
2020-04 데이터를 월별_데이터\2020-04.xlsx에 저장했습니다.
2020-05 데이터를 월별_데이터\2020-05.xlsx에 저장했습니다.
2020-06 데이터를 월별_데이터\2020-06.xlsx에 저장했습니다.
2020-07 데이터를 월별_데이터\2020-07.xlsx에 저장했습니다.
2020-08 데이터를 월별_데이터\2020-08.xlsx에 저장했습니다.
2020-09 데이터를 월별_데이터\2020-09.xlsx에 저장했습니다.
2020-10 데이터를 월별_데이터\2020-10.xlsx에 저장했습니다.
2020-11 데이터를 월별_데이터\2020-11.xlsx에 저장했습니다.
2020-12 데이터를 월별_데이터\2020-12.xlsx에 저장했습니다.
2021-01 데이터를 월별_데이터\2021-01.xlsx에 저장했습니다.
2021-02 데이터를 월별_데이터\2021-02.xlsx에 저장했습니다.
2021-03 데이터를 월별_데이터\2021-03.xlsx에 저장했습니다.
2021-04 데이터를 월별_데이터\2021-04.xlsx에 저장했습니다.
2021-05 데이터를 월별_데이터\2021-05.xlsx에 저장했습니다.
2021-06 데이터를 월별_데이터\2021-06.xlsx에 저장했습니다.
2021-07 데이터를 월별_데이터\2021-07.xlsx에 저장했습니다.
2021-08 데이터를 월별_데이터\2021-08.xlsx에 저장했습니다.
2021-09 데이터를 월별_데이터\2021-09.xlsx에 저장했습니다.
2021-10 데이터를 월별_데이터\2021-10.xlsx에 저장했습니다.
2021-11 데이터를 월별_데이터\2021-11.xlsx에 저장했습니다.
2021-12 데이터를 월별_데이터\2021-12.xlsx에 

In [13]:
df5 = read_excel("월별_데이터/2020-01.xlsx")
df5

,회원카드번호,연월,등록명,개월수,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
0,17030054,2020-01,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20010025,2020-01,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13110097,2020-01,헬스,12,15.0,19.0,2.0,0.0,15.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18050033,2020-01,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15070020,2020-01,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,19010007,2020-01,헬스,12,9.0,14.0,13.0,10.0,12.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,20010078,2020-01,헬스,12,1.0,14.0,14.0,5.0,12.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,19010076,2020-01,골프,12,4.0,14.0,6.0,12.0,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,18080017,2020-01,헬스,12,22.0,20.0,9.0,0.0,18.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df6 = df5.dropna(subset=["등록명"])
df6

,회원카드번호,연월,등록명,개월수,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
2,13110097,2020-01,헬스,12,15.0,19.0,2.0,0.0,15.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20010027,2020-01,헬스,10,4.0,2.0,1.0,6.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,18030043,2020-01,헬스,12,11.0,14.0,0.0,0.0,0.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20010004,2020-01,골프,10,5.0,3.0,8.0,1.0,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,20010024,2020-01,헬스,10,14.0,15.0,0.0,0.0,2.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,19010007,2020-01,헬스,12,9.0,14.0,13.0,10.0,12.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,20010078,2020-01,헬스,12,1.0,14.0,14.0,5.0,12.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,19010076,2020-01,골프,12,4.0,14.0,6.0,12.0,9.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,18080017,2020-01,헬스,12,22.0,20.0,9.0,0.0,18.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df7 = df6.dropna(axis=1, how='all')
df7

,회원카드번호,연월,등록명,개월수,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02
2,13110097,2020-01,헬스,12,15.0,19.0,2.0,0.0,15.0,18.0,18.0,10.0,0.0,2.0,10.0,0.0,NaN,NaN
5,20010027,2020-01,헬스,10,4.0,2.0,1.0,6.0,3.0,2.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
6,18030043,2020-01,헬스,12,11.0,14.0,0.0,0.0,0.0,19.0,20.0,6.0,0.0,0.0,4.0,0.0,NaN,NaN
8,20010004,2020-01,골프,10,5.0,3.0,8.0,1.0,8.0,8.0,11.0,4.0,0.0,0.0,NaN,NaN,NaN,NaN
11,20010024,2020-01,헬스,10,14.0,15.0,0.0,0.0,2.0,8.0,10.0,8.0,5.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,19010007,2020-01,헬스,12,9.0,14.0,13.0,10.0,12.0,13.0,15.0,16.0,3.0,0.0,0.0,0.0,NaN,NaN
202,20010078,2020-01,헬스,12,1.0,14.0,14.0,5.0,12.0,10.0,10.0,6.0,10.0,14.0,7.0,0.0,NaN,NaN
203,19010076,2020-01,골프,12,4.0,14.0,6.0,12.0,9.0,8.0,5.0,7.0,0.0,8.0,3.0,0.0,NaN,NaN
204,18080017,2020-01,헬스,12,22.0,20.0,9.0,0.0,18.0,19.0,24.0,20.0,9.0,13.0,9.0,0.0,NaN,NaN


In [28]:
df7['개월수'].unique()

array([12, 10,  6, 14,  5,  3,  0,  7], dtype=int64)

In [38]:
df8 = df7.drop(df7[df7['개월수'] == 0].index, axis=0)
df8

,회원카드번호,연월,등록명,개월수,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02
2,13110097,2020-01,헬스,12,15.0,19.0,2.0,0.0,15.0,18.0,18.0,10.0,0.0,2.0,10.0,0.0,NaN,NaN
5,20010027,2020-01,헬스,10,4.0,2.0,1.0,6.0,3.0,2.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
6,18030043,2020-01,헬스,12,11.0,14.0,0.0,0.0,0.0,19.0,20.0,6.0,0.0,0.0,4.0,0.0,NaN,NaN
8,20010004,2020-01,골프,10,5.0,3.0,8.0,1.0,8.0,8.0,11.0,4.0,0.0,0.0,NaN,NaN,NaN,NaN
11,20010024,2020-01,헬스,10,14.0,15.0,0.0,0.0,2.0,8.0,10.0,8.0,5.0,3.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,19010007,2020-01,헬스,12,9.0,14.0,13.0,10.0,12.0,13.0,15.0,16.0,3.0,0.0,0.0,0.0,NaN,NaN
202,20010078,2020-01,헬스,12,1.0,14.0,14.0,5.0,12.0,10.0,10.0,6.0,10.0,14.0,7.0,0.0,NaN,NaN
203,19010076,2020-01,골프,12,4.0,14.0,6.0,12.0,9.0,8.0,5.0,7.0,0.0,8.0,3.0,0.0,NaN,NaN
204,18080017,2020-01,헬스,12,22.0,20.0,9.0,0.0,18.0,19.0,24.0,20.0,9.0,13.0,9.0,0.0,NaN,NaN


In [40]:
# '개월수' 열의 고유한 값들을 가져와서 정렬
unique_months = sorted(df8['개월수'].unique())

# 각 '개월수'에 해당하는 데이터를 저장할 딕셔너리 생성
month_groups = {}

# '개월수' 열의 고유한 값들에 따라 데이터를 그룹화하여 딕셔너리에 저장
for month in unique_months:
    month_groups[month] = df8[df8['개월수'] == month]

# 각 '개월수' 그룹에 해당하는 데이터 출력
for month, group_data in month_groups.items():
    print(f"개월수 {month}에 해당하는 데이터:")
    print(group_data)
    print()

개월수 3에 해당하는 데이터:
       회원카드번호       연월 등록명  개월수  2020-01  2020-02  2020-03  2020-04  2020-05  \
73   19040035  2020-01  헬스    3      4.0     12.0      5.0      NaN      NaN   
179  17050063  2020-01  골프    3     10.0      9.0      6.0      NaN      NaN   

     2020-06  2020-07  2020-08  2020-09  2020-10  2020-11  2020-12  2021-01  \
73       NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
179      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

     2021-02  
73       NaN  
179      NaN  

개월수 5에 해당하는 데이터:
      회원카드번호       연월 등록명  개월수  2020-01  2020-02  2020-03  2020-04  2020-05  \
45  14110071  2020-01  헬스    5     21.0     21.0      7.0      0.0      0.0   

    2020-06  2020-07  2020-08  2020-09  2020-10  2020-11  2020-12  2021-01  \
45      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   

    2021-02  
45      NaN  

개월수 6에 해당하는 데이터:
       회원카드번호       연월 등록명  개월수  2020-01  2020-02  2020-03  2020-04  2020-05  \

In [44]:

# '개월수' 열의 고유한 값들을 가져와서 정렬
unique_months = sorted(df8['개월수'].unique())

# 각 '개월수'에 해당하는 데이터를 저장할 딕셔너리 생성
month_groups = {}

# '개월수' 열의 고유한 값들에 따라 데이터를 그룹화하여 딕셔너리에 저장
for month in unique_months:
    month_groups[month] = df8[df8['개월수'] == month]


# 각 '개월수' 그룹에 해당하는 데이터를 엑셀 파일로 저장
for month, group_data in month_groups.items():
    # 엑셀 파일명 설정 (파일명은 '개월수_월.xlsx' 형식)
    파일명 = f"{month}개월등록.xlsx"
    # 데이터프레임을 엑셀 파일로 저장
    group_data.to_excel(파일명, index=False)
    print(f"개월수 {month}에 해당하는 데이터를 {파일명}에 저장했습니다.")

개월수 3에 해당하는 데이터를 3개월등록.xlsx에 저장했습니다.
개월수 5에 해당하는 데이터를 5개월등록.xlsx에 저장했습니다.
개월수 6에 해당하는 데이터를 6개월등록.xlsx에 저장했습니다.
개월수 7에 해당하는 데이터를 7개월등록.xlsx에 저장했습니다.
개월수 10에 해당하는 데이터를 10개월등록.xlsx에 저장했습니다.
개월수 12에 해당하는 데이터를 12개월등록.xlsx에 저장했습니다.
개월수 14에 해당하는 데이터를 14개월등록.xlsx에 저장했습니다.


In [94]:
# 파일 경로
월별_데이터_폴더 = Path("월별_데이터")

# 고유한 '개월수' 값을 기준으로 데이터를 처리하는 함수
def process_data(df):
    df = df.dropna(subset=["등록명"]).dropna(axis=1, how='all')
    df = df[df['개월수'] > 0]  # 음수 값을 가진 개월수는 제외합니다.
    return df

# 각 '개월수'에 해당하는 데이터를 저장하는 함수
def save_data_by_month(df):
    unique_months = sorted(df['개월수'].unique())
    month_groups = {}
    
    for month in unique_months:
        month_groups[month] = df[df['개월수'] == month]
    
    for month, group_data in month_groups.items():
        파일명 = 월별_데이터_폴더 / f"{month}개월등록.xlsx"
        group_data.to_excel(파일명, index=False)
        print(f"개월수 {month}에 해당하는 데이터를 {파일명}에 저장했습니다.")

# 월별 데이터 파일을 읽어 처리
for 파일 in 월별_데이터_폴더.glob("2023-12.xlsx"):
    df = pd.read_excel(파일)
    df = process_data(df)
    save_data_by_month(df)

개월수 1에 해당하는 데이터를 월별_데이터\1개월등록.xlsx에 저장했습니다.
개월수 6에 해당하는 데이터를 월별_데이터\6개월등록.xlsx에 저장했습니다.
개월수 10에 해당하는 데이터를 월별_데이터\10개월등록.xlsx에 저장했습니다.
개월수 12에 해당하는 데이터를 월별_데이터\12개월등록.xlsx에 저장했습니다.
개월수 24에 해당하는 데이터를 월별_데이터\24개월등록.xlsx에 저장했습니다.


In [95]:
# 빈 데이터프레임을 생성합니다.
result = pd.DataFrame()

# 월별 데이터 디렉토리 경로
data_dir = "월별_데이터"

# 월별 데이터 디렉토리 안의 모든 파일에 대해 반복합니다.
for file_name in os.listdir(data_dir):
    if re.match(r"\d+개월등록.xlsx", file_name):  # "n개월등록.xlsx" 형식에 맞는 파일만 선택합니다.
        file_path = os.path.join(data_dir, file_name)
        
        # 파일을 읽어옵니다.
        df = pd.read_excel(file_path)
        
        # 데이터 전처리 및 계산을 수행합니다.
        df = df.dropna(axis=1)
        df_transposed = df.transpose()
        
        print("열 이름 확인:", df_transposed.columns)  # 열 이름을 출력하여 확인합니다.
        
        trend_values = []
        for column_index, column in enumerate(df_transposed.columns[:]):
            x_values = np.arange(1, len(df_transposed[column]) + 1)
            y_values = [value for value in df_transposed[column].values if isinstance(value, (int, float)) and len(str(value)) < 3]
            slope, _, _, _, _ = linregress(x_values[:len(y_values)], y_values)
            trend_values.append(slope)
        
        df_transposed.loc['추세값'] = trend_values
        df_transposed = df_transposed.transpose()
        
        # 필요한 열만 선택합니다.
        df_selected = df_transposed[['회원카드번호', '연월', '추세값']]
        
        # 결과 데이터프레임에 추가합니다.
        result = pd.concat([result, df_selected])

result.to_excel('2023-12-Plus.xlsx',index=False)

열 이름 확인: RangeIndex(start=0, stop=7, step=1)
열 이름 확인: RangeIndex(start=0, stop=24, step=1)
열 이름 확인: RangeIndex(start=0, stop=2, step=1)
열 이름 확인: RangeIndex(start=0, stop=1, step=1)
열 이름 확인: RangeIndex(start=0, stop=1, step=1)


In [99]:
# 월별 데이터 폴더 경로
월별_데이터_폴더 = Path("월별_데이터")

# 데이터프레임을 저장할 빈 리스트 생성
dfs = []

# 2020년부터 2023년까지의 "Plus"로 끝나는 파일을 읽어서 리스트에 추가
for year in range(2020, 2024):
    for month in range(1, 13):
        파일명 = f"{year}-{month:02d}-Plus.xlsx"
        if 월별_데이터_폴더 / 파일명 in 월별_데이터_폴더.glob("*-Plus.xlsx"):
            df = pd.read_excel(월별_데이터_폴더 / 파일명)
            dfs.append(df)

# 데이터프레임 리스트를 하나의 데이터프레임으로 병합
merged_df = pd.concat(dfs, ignore_index=True)

# 데이터프레임을 하나의 엑셀 파일로 저장
merged_df.to_excel("2020-2023-Plus.xlsx", index=False)